
# Hypothesis Test 3 - Days per Week

## Question

Is there a statistically significant difference in the mean percentage of A's depending on the number of days a class meets a week?

## Hypotheses

Null Hypothesis: There is no significant difference in the mean percentage of A's if a class meets one day a week or 2-5 days a week. 

($\mu$1 = $\mu$2 = $\mu$3 = $\mu$4 = $\mu$5)

Alternate Hypothesis: There is a significant difference in the mean percentage of A's if a class meets one day a week or 2-5 days a week. 

($\mu$1 != $\mu$2 != $\mu$3 != $\mu$4 != $\mu$5)


## Methodology

We select an alpha value of 0.05.

First, we will perform a one-way F-statistic test first to confirm there is a significance between the five groups.
If there is a significant difference, we will look first at a two-tail, two-sample t-test on each pairing of the days to see if there is a significant difference between two specific days. Finally, if the two sample test does how significance we will perform more testing to see if one day performs better than another day (a one-tailed, two-sample, t-test).

### Import Necessary Libraries

In [30]:
import math
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from simulations import simulate_days_ttests_with_samples, get_sample_size, clean_df

np.random.seed(2019)

## Data Preparation

### Connect to UW-Madison PostgreSQL database

In [31]:
conn = psycopg2.connect("host=localhost,dbname=postgres,user=postgres")

cur = conn.cursor()

We are interested in the percentage of A's given in a specific course and section. While there are other grades such as "credit" and "incomplete", these grades are not given a GPA value and we will therefore ignore them and only use A-F grades. We also want to collect the days a course/section meets so we can get the total number of days that a class meets. Finally, we want the section type as we want to make sure that we measure Lecture and Seminar classes as opposed to Independent/Field Studies and Labs/Discussions (which often contribute to a lecture class' grade).


In [32]:
group_and_as = """
SELECT 
  grade.a_count
, grade.ab_count
, grade.b_count
, grade.bc_count
, grade.c_count
, grade.d_count
, grade.f_count
, sched.mon
, sched.tues
, sched.wed
, sched.thurs
, sched.fri
, sched.sat
, sched.sun
, sect.section_type
, grade.course_offering_uuid
, grade.section_number
FROM 
  sections sect
INNER JOIN 
  schedules sched 
ON 
  sched.uuid = sect.schedule_uuid
INNER JOIN 
  grade_distributions grade 
ON 
  sect.number = grade.section_number 
AND 
  sect.course_offering_uuid = grade.course_offering_uuid;
"""

In [33]:
cur.execute(group_and_as)
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217399 entries, 0 to 217398
Data columns (total 17 columns):
a_count                 217399 non-null int64
ab_count                217399 non-null int64
b_count                 217399 non-null int64
bc_count                217399 non-null int64
c_count                 217399 non-null int64
d_count                 217399 non-null int64
f_count                 217399 non-null int64
mon                     217399 non-null bool
tues                    217399 non-null bool
wed                     217399 non-null bool
thurs                   217399 non-null bool
fri                     217399 non-null bool
sat                     217399 non-null bool
sun                     217399 non-null bool
section_type            217399 non-null object
course_offering_uuid    217399 non-null object
section_number          217399 non-null int64
dtypes: bool(7), int64(8), object(2)
memory usage: 18.0+ MB


Our original dataframe has 217,399 entries.

In [34]:
# remove non-LEC/SEM, add total count and remove if zero
df = clean_df(df) 
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88368 entries, 48 to 217361
Data columns (total 18 columns):
a_count                 88368 non-null int64
ab_count                88368 non-null int64
b_count                 88368 non-null int64
bc_count                88368 non-null int64
c_count                 88368 non-null int64
d_count                 88368 non-null int64
f_count                 88368 non-null int64
mon                     88368 non-null bool
tues                    88368 non-null bool
wed                     88368 non-null bool
thurs                   88368 non-null bool
fri                     88368 non-null bool
sat                     88368 non-null bool
sun                     88368 non-null bool
section_type            88368 non-null object
course_offering_uuid    88368 non-null object
section_number          88368 non-null int64
total_count             88368 non-null int64
dtypes: bool(7), int64(9), object(2)
memory usage: 8.7+ MB


In [35]:
df.head()

a_count  ab_count  b_count  bc_count  c_count  d_count  f_count   mon  \
48       16        44       33         4        5        0        0  True   
49        2         5        5         1        0        0        0  True   
50        2         3        5         2        0        0        0  True   
51        4         5        2         1        1        0        0  True   
52        1         7        2         0        0        0        0  True   

     tues   wed  thurs    fri    sat    sun section_type  \
48  False  True  False  False  False  False          LEC   
49  False  True  False   True  False  False          LEC   
50  False  True  False   True  False  False          LEC   
51  False  True  False   True  False  False          LEC   
52  False  True  False   True  False  False          LEC   

                    course_offering_uuid  section_number  total_count  
48  a7e2605c-df99-31e1-86c5-05f4f6d3d463               1          102  
49  36754980-48d2-3b34-ad0c-d621a6cb30df              32           13  
50  36754980-48d2-3b34-ad0c-d621a6cb30df              48           12  
51  36754980-48d2-3b34-ad0c-d621a6cb30df              23           13  
52  36754980-48d2-3b34-ad0c-d621a6cb30df              27           10

Now there are 88,368 entries.

In [36]:
df['days_a_week'] = sum([df['mon'], df['tues'], df['wed'], df['thurs'], df['fri'], df['sat'], df['sun']])

In [37]:
df.head()

a_count  ab_count  b_count  bc_count  c_count  d_count  f_count   mon  \
48       16        44       33         4        5        0        0  True   
49        2         5        5         1        0        0        0  True   
50        2         3        5         2        0        0        0  True   
51        4         5        2         1        1        0        0  True   
52        1         7        2         0        0        0        0  True   

     tues   wed  thurs    fri    sat    sun section_type  \
48  False  True  False  False  False  False          LEC   
49  False  True  False   True  False  False          LEC   
50  False  True  False   True  False  False          LEC   
51  False  True  False   True  False  False          LEC   
52  False  True  False   True  False  False          LEC   

                    course_offering_uuid  section_number  total_count  \
48  a7e2605c-df99-31e1-86c5-05f4f6d3d463               1          102   
49  36754980-48d2-3b34-ad0c-d621a6cb30df              32           13   
50  36754980-48d2-3b34-ad0c-d621a6cb30df              48           12   
51  36754980-48d2-3b34-ad0c-d621a6cb30df              23           13   
52  36754980-48d2-3b34-ad0c-d621a6cb30df              27           10   

    days_a_week  
48            2  
49            3  
50            3  
51            3  
52            3

In [38]:
df.sort_values(by='days_a_week', ascending=False).head()

a_count  ab_count  b_count  bc_count  c_count  d_count  f_count   mon  \
131686        5         1        0         0        0        0        0  True   
156580      136        25        7         0        0        0        0  True   
11114        10         3        0         0        0        0        0  True   
175475       61        62       26        13        0        0        0  True   
11949        59        56       29         7        0        0        0  True   

        tues   wed  thurs   fri   sat    sun section_type  \
131686  True  True   True  True  True   True          LEC   
156580  True  True   True  True  True  False          LEC   
11114   True  True   True  True  True  False          LEC   
175475  True  True   True  True  True  False          LEC   
11949   True  True   True  True  True  False          LEC   

                        course_offering_uuid  section_number  total_count  \
131686  27aea5aa-a687-3963-be2c-3d0cd3065fd1               3            6   
156580  d39c3ff3-080b-3184-9cdb-52d28a2f65fb               1          168   
11114   50df035b-45a0-3cc6-89aa-b2838e192917              16           13   
175475  cc9e03d4-93cd-35ed-a570-a348048a7d47               1          162   
11949   105e20eb-f63b-30e7-b5a5-0ba6e45f61a4               1          151   

        days_a_week  
131686            7  
156580            6  
11114             6  
175475            6  
11949             6

In [39]:
df.sort_values(by='days_a_week', ascending=True).head()

a_count  ab_count  b_count  bc_count  c_count  d_count  f_count  \
167966       28        45       90        37       18        2        2   
74189         4         1        2         0        0        0        0   
178755       17         1        6         0        4        0        0   
4371         54        49       49        17       17        3        4   
4372         54        49       49        17       17        3        4   

          mon   tues    wed  thurs    fri    sat    sun section_type  \
167966  False  False  False  False  False  False  False          LEC   
74189   False  False  False  False  False  False  False          LEC   
178755  False  False  False  False  False  False  False          LEC   
4371    False  False  False  False  False  False  False          LEC   
4372    False  False  False  False  False  False  False          LEC   

                        course_offering_uuid  section_number  total_count  \
167966  bf034c00-9141-388c-83a4-ad5ad50be94c              10          222   
74189   d8cb9cd0-29bb-3609-a0fe-0717a851247f               5            7   
178755  f4bef7c1-cff9-31fd-8ae5-b0090e5a468e               7           28   
4371    0346e9c4-fb6b-3056-a2d3-60cb08960273               1          193   
4372    0346e9c4-fb6b-3056-a2d3-60cb08960273               1          193   

        days_a_week  
167966            0  
74189             0  
178755            0  
4371              0  
4372              0

We will drop classes that meet 0, 6, or 7 days a week as we only want to schedule classes on a standard 5 day week.

In [40]:
df = df[df['days_a_week'] < 6]
df = df[df['days_a_week'] > 0]

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85084 entries, 48 to 217361
Data columns (total 19 columns):
a_count                 85084 non-null int64
ab_count                85084 non-null int64
b_count                 85084 non-null int64
bc_count                85084 non-null int64
c_count                 85084 non-null int64
d_count                 85084 non-null int64
f_count                 85084 non-null int64
mon                     85084 non-null bool
tues                    85084 non-null bool
wed                     85084 non-null bool
thurs                   85084 non-null bool
fri                     85084 non-null bool
sat                     85084 non-null bool
sun                     85084 non-null bool
section_type            85084 non-null object
course_offering_uuid    85084 non-null object
section_number          85084 non-null int64
total_count             85084 non-null int64
days_a_week             85084 non-null int64
dtypes: bool(7), int64(10), object(2)
m

After dropping these rows we are left with 85,084 courses/sections.

Next, we create a copy of our data frame.

In [42]:
df2 = df.copy()

Create a new column that represents the percentage of A's in the course/section.

In [43]:
df2['ratio_of_as'] = df['a_count'] / df['total_count']

We want to ensure that every row is unique (has a unique combination of `course_offering_uuid` and `section_number`) so we will drop duplicates if there are any.

In [44]:
df2.drop_duplicates(inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71381 entries, 48 to 217360
Data columns (total 20 columns):
a_count                 71381 non-null int64
ab_count                71381 non-null int64
b_count                 71381 non-null int64
bc_count                71381 non-null int64
c_count                 71381 non-null int64
d_count                 71381 non-null int64
f_count                 71381 non-null int64
mon                     71381 non-null bool
tues                    71381 non-null bool
wed                     71381 non-null bool
thurs                   71381 non-null bool
fri                     71381 non-null bool
sat                     71381 non-null bool
sun                     71381 non-null bool
section_type            71381 non-null object
course_offering_uuid    71381 non-null object
section_number          71381 non-null int64
total_count             71381 non-null int64
days_a_week             71381 non-null int64
ratio_of_as             71381 non-null 

Dropping duplicates has brought our number of courses/sections to 71,381.

In [45]:
sns.jointplot('days_a_week', 'ratio_of_as',df2)
plt.title("Days a Week and Percentage of A's at UW-Madison", loc='left')
plt.show();

/Users/kaylischulz/anaconda3/envs/learn-env/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


The scatterplot itself does not tell us too much, but if we look at the histograms we can see the most classes meet 2 days a week followed by one day a week. We can also see that a very large majority of classes have very high (almost 1.0) percentage of students that receive an A.

## Modeling/Testing

### One-Way ANOVA

Create new columns for one-hot encoding for days a class meets a week in order to perform a one-way ANOVA test.

In [46]:
dummied_days = pd.get_dummies(df2, columns=['days_a_week'])

In [47]:
dummied_days.head()

a_count  ab_count  b_count  bc_count  c_count  d_count  f_count   mon  \
48       16        44       33         4        5        0        0  True   
49        2         5        5         1        0        0        0  True   
50        2         3        5         2        0        0        0  True   
51        4         5        2         1        1        0        0  True   
52        1         7        2         0        0        0        0  True   

     tues   wed      ...        section_type  \
48  False  True      ...                 LEC   
49  False  True      ...                 LEC   
50  False  True      ...                 LEC   
51  False  True      ...                 LEC   
52  False  True      ...                 LEC   

                    course_offering_uuid  section_number  total_count  \
48  a7e2605c-df99-31e1-86c5-05f4f6d3d463               1          102   
49  36754980-48d2-3b34-ad0c-d621a6cb30df              32           13   
50  36754980-48d2-3b34-ad0c-d621a6cb30df              48           12   
51  36754980-48d2-3b34-ad0c-d621a6cb30df              23           13   
52  36754980-48d2-3b34-ad0c-d621a6cb30df              27           10   

   ratio_of_as days_a_week_1  days_a_week_2  days_a_week_3  days_a_week_4  \
48    0.156863             0              1              0              0   
49    0.153846             0              0              1              0   
50    0.166667             0              0              1              0   
51    0.307692             0              0              1              0   
52    0.100000             0              0              1              0   

    days_a_week_5  
48              0  
49              0  
50              0  
51              0  
52              0  

[5 rows x 24 columns]

In [48]:
formula = 'ratio_of_as~C(days_a_week_1)+C(days_a_week_2)+C(days_a_week_3)+C(days_a_week_4)+C(days_a_week_5)'
lm = ols(formula, dummied_days).fit()
table = sm.stats.anova_lm(lm, type=3)
print(table)

                       df       sum_sq      mean_sq             F  \
C(days_a_week_1)      1.0  1367.433258  1367.433258  20317.455858   
C(days_a_week_2)      1.0    12.173341    12.173341    180.872684   
C(days_a_week_3)      1.0     5.998375     5.998375     89.124439   
C(days_a_week_4)      1.0     3.960900     3.960900     58.851432   
C(days_a_week_5)      1.0     0.061488     0.061488      0.913591   
Residual          71375.0  4803.778066     0.067303           NaN   

                        PR(>F)  
C(days_a_week_1)  0.000000e+00  
C(days_a_week_2)  3.508406e-41  
C(days_a_week_3)  3.814301e-21  
C(days_a_week_4)  1.721825e-14  
C(days_a_week_5)  3.391665e-01  
Residual                   NaN  


Using the Bonferroni correction we adjust our alpha to reflect 5 independent variables (0.05 / 5). Our adjusted alpha is 0.01. Since looking at two-tailed we are looking at `PR(>F)`'s that are less than 0.005

We are comparing to 1 day a week.

The `PR(>F)` values are less than alpha for 2, 3, and 4 days which means we reject the null and there is a difference in the percentage of A's given whether the class meets 2 to 1, 3 to 1, or 4 to 1 days a week. The `PR(>F)` for five is greater than alpha so we fail to reject the null. There is not enough information to say that there is a difference between meeting 1 day a week and 5 days a week.

The most influential of 2, 3, and 4 appears to be 2 days a week as it has the smallest `PR(>F)` value.

### One-Way F-Statistic Test

For further confirmation, we check a one-way F-statistic test on the same data.

First, we narrow down our dataframe to just our dependent and independent variables.

In [49]:
days_and_as = pd.DataFrame({'days_a_week': df2['days_a_week'], 'ratio_of_as': df2['ratio_of_as']})

We then split that dataframe into separate dataframes depending on how many days a week the class meets.

In [50]:
ratios_of_as = {}
input_dict = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5}
for key, val in input_dict.items():
    ratios_of_as[key] = days_and_as.loc[days_and_as['days_a_week'] == val, 'ratio_of_as']

In [51]:
stats.f_oneway(ratios_of_as['one'], ratios_of_as['two'],
               ratios_of_as['three'], ratios_of_as['four'],
               ratios_of_as['five'])

F_onewayResult(statistic=5161.7029692405, pvalue=0.0)

The test tells us that there is a significant statistical difference in the mean percentage of A's by how many days a week a class meets, but it does not tell us in which pairing.

### Two-Tailed Two-Sample T-Test

As the number of combinations is small (only 10), we will run a two-tailed, two-sample t-test on each pairing to see which pairings of the five have also show statistical significance.

In [52]:
# Generate the pairs of days
days_pairs = []
days = [1, 2, 3, 4, 5]

for days1 in range(4):
    for days2  in range(days1+1,5):
        days_pairs.append((days[days1], days[days2]))

In [53]:
# Conduct t-test on each pair
for days1, days2 in days_pairs: 
    first = days_and_as.loc[days_and_as['days_a_week'] == days1, 'ratio_of_as']
    second = days_and_as.loc[days_and_as['days_a_week'] == days2, 'ratio_of_as']
    print(days1, days2)
    print(stats.ttest_ind(first, second)) 

1 2
Ttest_indResult(statistic=124.06730444574468, pvalue=0.0)
1 3
Ttest_indResult(statistic=114.51292840584208, pvalue=0.0)
1 4
Ttest_indResult(statistic=58.79479493159298, pvalue=0.0)
1 5
Ttest_indResult(statistic=33.41606962852344, pvalue=2.9967139670907445e-239)
2 3
Ttest_indResult(statistic=16.588715477715542, pvalue=1.2847619665460366e-61)
2 4
Ttest_indResult(statistic=4.502073298273141, pvalue=6.751122649942457e-06)
2 5
Ttest_indResult(statistic=-5.960003315452327, pvalue=2.547577395262071e-09)
3 4
Ttest_indResult(statistic=-4.46296307926901, pvalue=8.139153894994629e-06)
3 5
Ttest_indResult(statistic=-12.782980959959115, pvalue=3.257597015149319e-37)
4 5
Ttest_indResult(statistic=-8.030417363713108, pvalue=1.2344964015390695e-15)


Each of these pairings is statistically significant for two-tailed tests as all p-values are less than 0.025 (half of alpha). We will make new hypotheses to test each pairing.

### One-tailed Two-Sample T-Test

We will keep an alpha value of 0.05.

Of the pairings (1,2), (1,3), (1,4), (1,5), (2,3), (2,4), (2,5), (3,4), (3,5), (4,5):

Null hypothesis: The class that meets less in a pair does not have a statistically significant difference in mean percentage of A from the class the meets more times a week ($\mu$1 = $\mu$2).

Alternative: The class that meets less in a pair does have a statistically significant positive difference in mean percentage of A from the class the meets more times a week ($\mu$1 > $\mu$2).

#### Get Sample Size

In [54]:
moe = 0.1 # we chose 0.1 as this is a 10% difference in percentage of A's
z = 1.96 # z value for alpha 0.05

vars_and_n = {}
for key, val in ratios_of_as.items():
    vars_and_n[key] = (np.var(val), len(val))

n = get_sample_size(moe, z, vars_and_n)
n

52

#### Simulate t-tests with multiple samples

In [55]:
simulate_days_ttests_with_samples(days_pairs, ratios_of_as, seed=1, sample_size=n)

1 2
Ttest_indResult(statistic=5.997746334778467, pvalue=3.06521987687498e-08)
1 3
Ttest_indResult(statistic=7.8615158465573325, pvalue=4.071213868697429e-12)
1 4
Ttest_indResult(statistic=7.336956585725705, pvalue=5.4084228902392695e-11)
1 5
Ttest_indResult(statistic=4.944869727124291, pvalue=3.0050843252198843e-06)
2 3
Ttest_indResult(statistic=1.2165108685665096, pvalue=0.22659791459612708)
2 4
Ttest_indResult(statistic=0.8613316080426959, pvalue=0.3910757365811789)
2 5
Ttest_indResult(statistic=-1.0172950173432207, pvalue=0.3114200441953083)
3 4
Ttest_indResult(statistic=-0.3673355677892229, pvalue=0.7141297774123738)
3 5
Ttest_indResult(statistic=-2.33692605793659, pvalue=0.021397304901270755)
4 5
Ttest_indResult(statistic=-1.9553236395170128, pvalue=0.05328110280608556)


In [56]:
simulate_days_ttests_with_samples(days_pairs, ratios_of_as, seed=5, sample_size=n)

1 2
Ttest_indResult(statistic=6.2807973287718895, pvalue=8.351379714785528e-09)
1 3
Ttest_indResult(statistic=7.034518339158318, pvalue=2.350379932305796e-10)
1 4
Ttest_indResult(statistic=6.748869185829602, pvalue=9.247627898736811e-10)
1 5
Ttest_indResult(statistic=5.227834593732263, pvalue=9.153741893265495e-07)
2 3
Ttest_indResult(statistic=0.7127998190712176, pvalue=0.4775971381243108)
2 4
Ttest_indResult(statistic=0.2418768761624164, pvalue=0.8093611927818167)
2 5
Ttest_indResult(statistic=-0.8268411843698534, pvalue=0.4102571738710825)
3 4
Ttest_indResult(statistic=-0.4973164113089079, pvalue=0.6200362845391918)
3 5
Ttest_indResult(statistic=-1.5170129368523522, pvalue=0.13235666708238575)
4 5
Ttest_indResult(statistic=-1.0878957071902722, pvalue=0.2792050834513471)


In [57]:
simulate_days_ttests_with_samples(days_pairs, ratios_of_as, seed=10, sample_size=n)

1 2
Ttest_indResult(statistic=4.597722507827227, pvalue=1.2285716789110848e-05)
1 3
Ttest_indResult(statistic=7.834073272234275, pvalue=4.665994831323303e-12)
1 4
Ttest_indResult(statistic=6.054318694286133, pvalue=2.368467711423822e-08)
1 5
Ttest_indResult(statistic=4.989192017372916, pvalue=2.5003961864975634e-06)
2 3
Ttest_indResult(statistic=2.557501337255894, pvalue=0.012013929439670867)
2 4
Ttest_indResult(statistic=1.1039271095382506, pvalue=0.2722222584616423)
2 5
Ttest_indResult(statistic=0.36284335343212587, pvalue=0.7174727089316957)
3 4
Ttest_indResult(statistic=-1.514654376477978, pvalue=0.1329527022143946)
3 5
Ttest_indResult(statistic=-2.1627950443483868, pvalue=0.03289349710240017)
4 5
Ttest_indResult(statistic=-0.7219342300445417, pvalue=0.4719867064080897)


In [58]:
simulate_days_ttests_with_samples(days_pairs, ratios_of_as, seed=20, sample_size=n)

1 2
Ttest_indResult(statistic=5.160172648900121, pvalue=1.2201748983430053e-06)
1 3
Ttest_indResult(statistic=6.025297372085523, pvalue=2.703808692693925e-08)
1 4
Ttest_indResult(statistic=4.9671569118117995, pvalue=2.740022031223619e-06)
1 5
Ttest_indResult(statistic=3.711808451849479, pvalue=0.00033556699848977874)
2 3
Ttest_indResult(statistic=0.8918320842108677, pvalue=0.3745819924300039)
2 4
Ttest_indResult(statistic=-0.30514205958466895, pvalue=0.760879756725287)
2 5
Ttest_indResult(statistic=-1.2847643795927621, pvalue=0.20178463694049134)
3 4
Ttest_indResult(statistic=-1.217970886403939, pvalue=0.2260449785308498)
3 5
Ttest_indResult(statistic=-2.1368580052343202, pvalue=0.03500060487982182)
4 5
Ttest_indResult(statistic=-1.020018597179926, pvalue=0.3101328890851109)


Over four tests, we see the most significant pairings are with 1 class a week. This is because the p-value is always below our alpha value of 0.05. We then reject the null, and support the alternative that the one class a week has a greater mean ratio of A's than meeting 2-5 days a week. There are other pairings that sometimes show significance but are not consistent over many tests.

## Evaluation 
Having lecture/seminar classes once a week has a significantlly greater mean of percentages of A's than having class 2, 3, 4, or 5 times a week.